In [1]:
import pandas as pd
from tqdm import tqdm
import string, json, math
import pytrec_eval
import numpy as np

In [2]:
train_output = '/mnt/scratch/chenqu/msmarco/original/session_train.txt'
dev_output = '/mnt/scratch/chenqu/msmarco/original/session_dev.txt'
test_output = '/mnt/scratch/chenqu/msmarco/original/session_test.txt'

dataset = 'msmarco'

if dataset == 'aol':
    CLICKS = 'clicks'
    TITLE = 'title'
elif dataset == 'msmarco':
    CLICKS = 'candidates'
    TITLE = 'content'

In [11]:
query_count = 0
target_query_count = 0
target_query_lens, doc_lens = [], []
target_click_count = 0
session_lens = []
session_count = 0

with open(test_output) as fin:
    for line in tqdm(fin):
        session_count += 1
        session = json.loads(line.strip())
        queries = session['query']
        session_lens.append(len(queries))
        session_len = len(queries)
        for i, query_dict in enumerate(queries):
            query_count += 1
            if dataset == 'aol' or i == session_len - 1:
                target_query_count += 1
                clicks = query_dict[CLICKS]
                click_list = [click['label'] for click in clicks]
                total_clicks = sum(click_list)
                target_click_count += total_clicks
                target_query_lens.append(len(query_dict['text'].split()))
                for doc in clicks:
                    doc_lens.append(len(doc[TITLE].split()))        

5860it [00:03, 1764.77it/s]


In [12]:
print('# sessions', session_count)
print('# queries', query_count)
print('# target queries', target_query_count)
print('avg target queries per session', target_query_count / session_count)
print('avg target query length', np.average(target_query_lens))
print('avg doc length', np.average(doc_lens))
print('avg doc per target query', len(doc_lens) / target_query_count)
print('avg click per target query', target_click_count / target_query_count)
print('min/avg/median/max query per session', min(session_lens), np.average(session_lens), 
                          np.median(session_lens), max(session_lens))

# sessions 5860
# queries 39916
# target queries 5860
avg target queries per session 1.0
avg target query length 3.8349829351535836
avg doc length 65.32257679180887
avg doc per target query 10.0
avg click per target query 1.0482935153583617
min/avg/median/max query per session 5 6.81160409556314 6.0 37
